In [2]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.1 MB/s eta 0:00:00


In [3]:
pip install pinecone-client

In [4]:
#pinecone apikey
api_key=""

In [7]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [8]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

#initialize the pinecone client
pc=Pinecone(api_key=api_key)

In [9]:
#create the index

if index_name not in pc.list_indexes().names():
    pc.create_index(
    name =index_name,
        dimension=384, #dimension of dense vector (huggingface embedding (sentence transform by default conver any text into 384 dimension))
    metric='dotproduct', ##sparse value supported
    spec=ServerlessSpec(cloud='aws',region="us-east-1")

    )

In [10]:
index =pc.Index(index_name)
index

In [13]:
import os

#set hugging-face token
os.environ['HF_TOKEN']=''
from langchain_huggingface import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
#for sparse matrix (TF-IDF)

from pinecone_text.sparse import BM25Encoder # by default TFIDF

bm25_encoder =BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
retriever =PineconeHybridSearchRetriever(embeddings=embedding,sparse_encoder=bm25_encoder,index=index)

In [16]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7d35a68f71f0>, index=<pinecone.data.index.Index object at 0x7d36cf796cb0>)

In [17]:
retriever.add_texts(
    [
     "Im in US in 2024",
     "I was in Pune 2023",
     "I was in pune 2022"

    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
retriever.invoke("Where I'm recently")

[Document(page_content='Im in US in 2024'),
 Document(page_content='I was in Pune 2023'),
 Document(page_content='I was in pune 2022')]